In [1]:
import os
import random

os.environ['KERAS_BACKEND'] = "tensorflow"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
from keras.utils.io_utils import HDF5Matrix
import h5py
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [2]:
# DEFINES
TRAIN_PATH = 'food-images/food_c101_n10099_r64x64x3.h5'
TEST_PATH = 'food-images/food_test_c101_n1000_r64x64x3.h5'

IMAGE_CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 100
IMAGE_WIDTH, IMAGE_HEIGHT = 3, 3
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)


In [3]:
# GET DATA
X_train = HDF5Matrix(TRAIN_PATH,
                     'images')  # Shape (10099, 32, 32, 3) => [image_index][pixel_width_index][pixel_height_index][color_channel_index]
y_train = HDF5Matrix(TRAIN_PATH, 'category')  # Shape (1000, 101) => [image_index][one_hot_encoded_vector]
print("X_train shape: ", X_train.shape, " y_train shape: ", y_train.shape)

X_test = HDF5Matrix(TEST_PATH,
                    'images')  # Shape (10099, 32, 32, 3) => [image_index][pixel_width_index][pixel_height_index][color_channel_index]
y_test = HDF5Matrix(TEST_PATH, 'category')  # Shape (1000, 101) => [image_index][one_hot_encoded_vector]
print("X_test shape: ", X_test.shape, " y_test shape: ", y_test.shape)

X_train shape:  (10099, 64, 64, 3)  y_train shape:  (10099, 101)
X_test shape:  (1000, 64, 64, 3)  y_test shape:  (1000, 101)


In [4]:
sess = tf.InteractiveSession( )

In [5]:
#Easy operations
def weight_variable(shape):
    """
    Initialise weight varaibles with smalls values around 0
    """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """
    Generates bias of a given shape
    """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """"
     Downsamples the feature map by 2X
    """
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

In [6]:
#Parameters
x = tf.placeholder(tf.float32, [None, 64, 64, 3]) # input of shape [picture_index][pixel_index]
y_ = tf.placeholder(tf.float32, [None, 101]) # Correct answers [picture_index][answer_class_index]

In [7]:
# Model
with tf.name_scope('conv1'):
    # maps one 1 color image (3 channels) to 64 feature maps
    W_conv1 = weight_variable([3, 3, IMAGE_CHANNELS, 64])
    b_conv1 = bias_variable([64])
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    
tf.summary.tensor_summary('input', x)
tf.summary.tensor_summary('W_conv1', W_conv1)
    #tf.summary.tensor_summary('b_conv1', b_conv1)
    #tf.summary.image('h_conv1', h_conv1)


with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope('conv2'):
    # maps 64 feature maps to 128 feature maps
    W_conv2 = weight_variable([3, 3, 64, 128])
    b_conv2 = bias_variable([128])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    
tf.summary.tensor_summary('W_conv2', W_conv2)
    #tf.summary.tensor_summary('b_conv2', b_conv2)
    #tf.summary.image('h_conv2', h_conv2)


with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

with tf.name_scope('conv3'):
    # maps 128 feature maps to 256 feature maps
    W_conv3 = weight_variable([3, 3, 128, 256])  # INPUT 3 channels => OUTPUT 64 CHANNELS
    b_conv3 = bias_variable([256])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    
tf.summary.tensor_summary('W_conv3', W_conv3)
    #tf.summary.tensor_summary('b_conv3', b_conv3)
    #tf.summary.image('h_conv3', h_conv3)


with tf.name_scope('reshape'):
    h_reshape = tf.reshape(h_conv3, [-1, 16 * 16 * 256])

with tf.name_scope('fc1'):
    W_fc1 = weight_variable([16 * 16 * 256, 1024])
    b_fc1 = bias_variable([1024])

    h_fc1 = tf.nn.relu( tf.matmul(h_reshape, W_fc1) + b_fc1 )

with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 2048])
    b_fc2 = bias_variable([2048])

    h_fc2 = tf.nn.relu( tf.matmul(h_fc1, W_fc2) + b_fc2 )

with tf.name_scope('readout'):
    W_readout = weight_variable([2048, 101])
    b_readout = bias_variable([101])  # 101 categories

    y = tf.matmul(h_fc2, W_readout) + b_readout

In [8]:
# Define Cross entropy & stuff
loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
optimizer_adam = tf.train.AdamOptimizer(1e-5).minimize(loss_cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
#Execute the session
merged = tf.summary.merge_all()
log_writer = tf.summary.FileWriter('tensorboard-v2', sess.graph)
tf.global_variables_initializer().run()

for i in range(EPOCHS):
    #Choose randomly a batch of data
    int_start = random.randint(0, X_train.shape[0] - BATCH_SIZE )
    int_end = int_start + BATCH_SIZE
    batch = ( X_train[ int_start:int_end ], y_train[ int_start:int_end ] )

    #Run the training
    optimizer_adam.run(session=sess, feed_dict={x: batch[0], y_: batch[1]})

        
    #Find and print the accuracy
    #train_accuracy, summary = sess.run( [accuracy, merged], feed_dict={x: batch[0], y_: batch[1]} )
    #sess.run(merged,feed_dict=feed_dict(False))
    summary, train_accuracy = sess.run([merged, accuracy], feed_dict={x: batch[0], y_: batch[1]})
    #train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
    print('step %d, training accuracy %s\n' % (i, train_accuracy))

    #Evalaute the accuracy on some training set evry x epochs
    if i % 20 == 0:
        test_accuracy = accuracy.eval(feed_dict={x: X_test, y_: y_test})
        print('step %d, test accuracy %s\n' % (1, test_accuracy))
        
        log_writer.add_summary(summary, i)


#Final Accuracy
test_accuracy = accuracy.eval(feed_dict={x: X_test, y_: y_test})
print('Final test accuracy %g' % (test_accuracy))
log_writer.flush()
log_writer.close()

step 0, training accuracy 0.0

step 1, test accuracy 0.011

step 1, training accuracy 0.0

step 2, training accuracy 0.03125

step 3, training accuracy 0.0

step 4, training accuracy 0.0

step 5, training accuracy 0.0

step 6, training accuracy 0.0

step 7, training accuracy 0.0

step 8, training accuracy 0.03125

step 9, training accuracy 0.0

step 10, training accuracy 0.0

step 11, training accuracy 0.0

step 12, training accuracy 0.03125

step 13, training accuracy 0.03125

step 14, training accuracy 0.0

step 15, training accuracy 0.0

step 16, training accuracy 0.0

step 17, training accuracy 0.0

step 18, training accuracy 0.0

step 19, training accuracy 0.0

step 20, training accuracy 0.0

step 1, test accuracy 0.013

step 21, training accuracy 0.0

step 22, training accuracy 0.0

step 23, training accuracy 0.0

step 24, training accuracy 0.0

step 25, training accuracy 0.0

step 26, training accuracy 0.0

step 27, training accuracy 0.03125

step 28, training accuracy 0.03125

